In [2]:
import os
import sys

def detect_platform():
    if os.path.exists('/kaggle/input'):
        return 'kaggle'
    elif 'google.colab' in sys.modules:
        return 'colab'
    else:
        return 'local'

PLATFORM = detect_platform()
print(f"Running on: {PLATFORM}")

Running on: local


In [3]:
if PLATFORM == 'kaggle':
    # Kaggle: repo must be attached as a dataset or cloned

    os.system('git clone https://github.com/RudraShivm/Skin-Cancer-Detection.git /kaggle/working/skin-cancer-detection')
    REPO_DIR = '/kaggle/working/skin-cancer-detection'

elif PLATFORM == 'colab':
    # Colab: clone from GitHub
    # For private repo, use Colab secrets
    try:
        from google.colab import userdata
        token = userdata.get('GITHUB_TOKEN')
        clone_url = f'https://{token}@github.com/RudraShivm/Skin-Cancer-Detection.git'
    except Exception:
        clone_url = 'https://github.com/RudraShivm/Skin-Cancer-Detection.git'  # public
    
    os.system(f'git clone {clone_url} /content/skin-cancer-detection')
    REPO_DIR = '/content/skin-cancer-detection'

else:  # local
    import rootutils
    REPO_DIR = str(rootutils.find_root(search_from='.', indicator='.project-root'))

print(f"Repo directory: {REPO_DIR}")

# Change to repo directory and install
os.chdir(REPO_DIR)
print(f"Installing dependency packages...")
os.system('pip install -r requirements.txt -q')  # ← install packages
print(f"Installing the current project as an editable Python package...")
os.system('pip install -e . -q')                 # ← make src/ importable

# Add to Python path
sys.path.insert(0, REPO_DIR)

print("✅ Setup complete")

Repo directory: /home/rudrashivm/code/dl/Skin-Cancer-Detection
Installing dependency packages...
Installing the current project as an editable Python package...
✅ Setup complete


In [4]:
# PROJECT_ROOT is needed by rootutils in train.py
os.environ['PROJECT_ROOT'] = REPO_DIR

if PLATFORM == 'kaggle':
    # Kaggle secrets (set in Kaggle → Add-ons → Secrets)
    try:
        from kaggle_secrets import UserSecretsClient
        secrets = UserSecretsClient()
        os.environ['WANDB_API_KEY'] = secrets.get_secret('WANDB_API_KEY')
    except Exception:
        pass  # Secrets not available, skip

elif PLATFORM == 'colab':
    # Colab secrets (set in Colab → 🔑 sidebar)
    try:
        from google.colab import userdata
        os.environ['WANDB_API_KEY'] = userdata.get('WANDB_API_KEY')
    except Exception:
        pass

else:  # local
    # Local: load from .env file if it exists
    env_file = os.path.join(REPO_DIR, '.env')
    if os.path.exists(env_file):
        with open(env_file) as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#') and '=' in line:
                    key, val = line.split('=', 1)
                    os.environ[key.strip()] = val.strip()

import wandb
wandb.login()

print(f"env::PROJECT_ROOT: {os.environ.get('PROJECT_ROOT')}")

AuthenticationError: WANDB_API_KEY invalid: API key may only contain the letters A-Z, digits and underscores.

In [5]:
DATA_DIR = None
CHECKPOINT_DIR = None
OUTPUT_DIR = None

if PLATFORM == 'kaggle':
    # Kaggle mounts competition data at /kaggle/input/
    DATA_DIR = '/kaggle/input/competitions/isic-2024-challenge'
    
    # Outputs go to /kaggle/working/ (this is what gets saved)
    CHECKPOINT_DIR = '/kaggle/working/checkpoints'
    OUTPUT_DIR = '/kaggle/working/outputs'

elif PLATFORM == 'colab':
    # Option A: Data in Google Drive
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    DATA_DIR = '/content/drive/MyDrive/isic-2024-challenge'
    
    # Option B: Data downloaded to Colab's ephemeral disk
    # DATA_DIR = '/content/isic-2024-challenge'
    
    CHECKPOINT_DIR = f'{REPO_DIR}/checkpoints'
    OUTPUT_DIR = f'{REPO_DIR}/outputs'

else:  # local
    DATA_DIR = os.path.join(REPO_DIR, 'data', 'isic-2024-challenge')
    CHECKPOINT_DIR = os.path.join(REPO_DIR, 'checkpoints')
    OUTPUT_DIR = os.path.join(REPO_DIR, 'outputs')

# Create dirs if needed
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Data:        {DATA_DIR}")
print(f"Checkpoints: {CHECKPOINT_DIR}")
print(f"Outputs:     {OUTPUT_DIR}")

# Verify data exists
if os.path.exists(DATA_DIR):
    print(f"✅ Data found: {os.listdir(DATA_DIR)[:5]}")
else:
    print(f"⚠️  Data NOT found at {DATA_DIR}")

Data:        /home/rudrashivm/code/dl/Skin-Cancer-Detection/data/isic-2024-challenge
Checkpoints: /home/rudrashivm/code/dl/Skin-Cancer-Detection/checkpoints
Outputs:     /home/rudrashivm/code/dl/Skin-Cancer-Detection/outputs
✅ Data found: ['sample_submission.csv', 'test-image.hdf5', 'test-metadata.csv', 'train-image.hdf5', 'train-image']


In [ ]:
# ============================================================
# TRAINING CONFIGURATION
# ============================================================
# Override MODELS_TO_TRAIN, N_FOLDS, DATA_FRACTION, MAX_EPOCHS.
# Models: efficientnet_b0, efficientnetv2_l, convnext_large,
#         eva02_large, swin_large, convnext_tiny
# ============================================================

import warnings, os, glob
warnings.filterwarnings('ignore')
os.environ['PYTHONWARNINGS'] = 'ignore'
os.environ['LIGHTNING_DISABLE_UPGRADE_WARNINGS'] = '1'
os.environ['LITMODULE_DISABLE_UPGRADE_WARNINGS'] = '1'

# ---- OVERRIDE THESE ----
MODELS_TO_TRAIN = ["efficientnet_b0"]
N_FOLDS = 1        # set to 1 for a quick test
DATA_FRACTION = 1
MAX_EPOCHS = 10

print(f"Models: {MODELS_TO_TRAIN}")
print(f"Folds: {N_FOLDS}  |  Data fraction: {DATA_FRACTION}  |  Max epochs: {MAX_EPOCHS}")
print(f"Checkpoint dir: {CHECKPOINT_DIR}")
print(f"Output dir:     {OUTPUT_DIR}")

for model_name in MODELS_TO_TRAIN:
    experiment = f"isic_{model_name}"
    print(f"\n{'='*60}\nMODEL: {model_name}  (experiment={experiment})\n{'='*60}")

    for fold in range(N_FOLDS):
        print(f"\n--- Fold {fold}/{N_FOLDS-1} ---")

        # Persisted output dir for Hydra logs, WandB saves, etc.
        fold_output_dir = os.path.join(OUTPUT_DIR, model_name, f"fold_{fold}")
        os.makedirs(fold_output_dir, exist_ok=True)

        # Persisted checkpoint dir
        fold_ckpt_dir = os.path.join(CHECKPOINT_DIR, model_name, f"fold_{fold}")
        os.makedirs(fold_ckpt_dir, exist_ok=True)

        # -------------------------------------------------------------------
        # Hydra override notes:
        # - ++ forces override even when the config key uses ${} interpolation
        # - Absolute paths with slashes MUST be shell-quoted so Hydra doesn't
        #   interpret "/" as a config-group separator. We wrap them in single
        #   quotes which the shell passes through verbatim to Hydra.
        # - paths.log_dir is the root that output_dir and hydra.run.dir both
        #   interpolate from, so overriding it alone would redirect everything.
        #   We also set output_dir and dirpath explicitly for safety.
        # -------------------------------------------------------------------
        hydra_overrides = [
            f"data.data_dir={DATA_DIR}",
            f"experiment={experiment}",
            f"data.fold={fold}",
            f"data.data_fraction={DATA_FRACTION}",
            f"trainer.max_epochs={MAX_EPOCHS}",
            "trainer.accelerator=gpu",
            "extras.print_config=false",
            "extras.enforce_tags=false",
            "logger=wandb",
            # Quote paths so slashes are not misread as config group separators
            f"'++paths.log_dir={fold_output_dir}'",
            f"'++paths.output_dir={fold_output_dir}'",
            f"'++callbacks.model_checkpoint.dirpath={fold_ckpt_dir}'",
        ]

        if PLATFORM == 'kaggle':
            hydra_overrides += [
                "trainer.precision=16-mixed",
                "data.num_workers=4",
                "~callbacks.rich_progress_bar",
                "~callbacks.model_summary",
            ]
        elif PLATFORM == 'colab':
            hydra_overrides += [
                "trainer.precision=16-mixed",
                "data.num_workers=4",
            ]
        else:  # local
            hydra_overrides += ["debug=default"]

        override_str = ' '.join(hydra_overrides)
        cmd = f'python src/train.py {override_str}'
        print(f"Running:\n  {cmd}\n")
        ret = os.system(cmd)
        print(f"\nExit code: {ret}")

        # ---- Checkpoint verification ----
        # Check expected location
        ckpts_found = glob.glob(os.path.join(fold_ckpt_dir, "*.ckpt"))
        if ckpts_found:
            print(f"\n\u2705 Checkpoints at {fold_ckpt_dir}:")
            for c in ckpts_found:
                print(f"   {os.path.basename(c)}")
        else:
            print(f"\n\u26a0\ufe0f  No .ckpt at expected path: {fold_ckpt_dir}")
            # Search everywhere under /kaggle/working/ to find where they landed
            search_roots = [OUTPUT_DIR, CHECKPOINT_DIR, REPO_DIR]
            all_ckpts = []
            for root in search_roots:
                all_ckpts += glob.glob(os.path.join(root, "**", "*.ckpt"), recursive=True)
            if all_ckpts:
                print("  Found .ckpt files at unexpected locations:")
                for c in set(all_ckpts):
                    print(f"   {c}")
            else:
                print("  No .ckpt files found anywhere — check training output above for errors")

print(f"\n{'='*60}\nTRAINING COMPLETE — checkpoints in: {CHECKPOINT_DIR}\n{'='*60}")


Running:
  python src/train.py data.data_dir=/home/rudrashivm/code/dl/Skin-Cancer-Detection/data/isic-2024-challenge experiment=isic_efficientnet_b0 trainer.accelerator=gpu extras.print_config=false extras.enforce_tags=false logger=wandb model.lr=0.00001 debug=default



Seed set to 42


[2026-02-18 09:18:21,060][__main__][INFO] - [rank: 0] Instantiating datamodule <src.data.isic_datamodule.ISICDataModule>
[2026-02-18 09:18:21,675][h5py._conv][DEBUG] - Creating converter from 7 to 5
[2026-02-18 09:18:21,676][h5py._conv][DEBUG] - Creating converter from 5 to 7
[2026-02-18 09:18:21,676][h5py._conv][DEBUG] - Creating converter from 7 to 5
[2026-02-18 09:18:21,676][h5py._conv][DEBUG] - Creating converter from 5 to 7
[2026-02-18 09:18:23,498][__main__][INFO] - [rank: 0] Instantiating model <src.models.isic_module.ISICLitModule>
[2026-02-18 09:18:26,346][timm.models._builder][INFO] - Loading pretrained weights from Hugging Face hub (timm/tf_efficientnet_b0.ns_jft_in1k)
[2026-02-18 09:18:26,370][httpcore.connection][DEBUG] - connect_tcp.started host='huggingface.co' port=443 local_address=None timeout=10 socket_options=None
[2026-02-18 09:18:26,526][httpcore.connection][DEBUG] - connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7fde961fe150>
[2

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Ti Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name           ┃ Type              ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model          │ EfficientNet      │  4.0 M │ train │     0 │
│ 1 │ criterion      │ BCEWithLogitsLoss │      0 │ train │     0 │
│ 2 │ train_auroc    │ BinaryAUROC       │      0 │ train │     0 ��
│ 3 │ val_auroc      │ BinaryAUROC       │      0 │ train │     0 │
│ 4 │ test_auroc     │ BinaryAUROC       │      0 │ train │     0 │
│ 5 │ train_loss     │ MeanMetric        │      0 │ train │     0 │
│ 6 │ val_loss       │ MeanMetric        │      0 │ train │     0 │
│ 7 │ test_loss      │ MeanMetric        │      0 │ train │     0 │
│ 8 │ val_auroc_best │ MaxMetric         │      0 │ train │     0 │
└───┴────────────────┴───────────────────┴────────┴───────┴───────┘
Trainable params: 4.0 M                                                         
Non-trainable params: 0           


Detected KeyboardInterrupt, attempting graceful shutdown ...


Epoch 0/0  ━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━ 33/102 0:00:12 • 0:00:25 2.85it/s 
[2026-02-18 09:18:50,846][src.utils.utils][INFO] - [rank: 0] Output dir: /home/rudrashivm/code/dl/Skin-Cancer-Detection/logs
[2026-02-18 09:18:50,848][httpcore.connection][DEBUG] - close.started
[2026-02-18 09:18:50,849][httpcore.connection][DEBUG] - close.complete


256

In [ ]:
if PLATFORM == 'kaggle':
    # Kaggle auto-saves /kaggle/working/ when notebook finishes
    # Just print where to find them
    print(f"Checkpoints saved to: {CHECKPOINT_DIR}")
    print("Kaggle will persist these when you save the notebook.")
    
    # Optional: commit results back to GitHub
    push_to_github = False 
    if push_to_github:
        from kaggle_secrets import UserSecretsClient
        secrets = UserSecretsClient()
        token    = secrets.get_secret('GITHUB_TOKEN')
        email    = secrets.get_secret('GIT_EMAIL')
        name     = secrets.get_secret('GIT_NAME')
        
        os.system(f'git config user.email "{email}"')
        os.system(f'git config user.name  "{name}"')
        os.system(f'git remote set-url origin https://{token}@github.com/YOU/REPO.git')
        
        os.system('git add outputs/ checkpoints/')
        os.system('git commit -m "Training results from Kaggle"')
        os.system('git push')
        print("✅ Pushed to GitHub")
    else:
        print(f"Results in /kaggle/working/ (auto-saved by Kaggle)")

elif PLATFORM == 'colab':
    # Option A: Save to Google Drive (persists across sessions)
    save_to_drive = True
    if save_to_drive:
        import shutil
        drive_save_dir = '/content/drive/MyDrive/isic-results'
        os.makedirs(drive_save_dir, exist_ok=True)
        shutil.copytree(CHECKPOINT_DIR, f'{drive_save_dir}/checkpoints', dirs_exist_ok=True)
        print(f"✅ Saved to Drive: {drive_save_dir}")
    
    # Option B: Download to local machine
    # from google.colab import files
    # files.download(f'{CHECKPOINT_DIR}/best.ckpt')

else:  # local
    # Already on local machine, nothing special needed
    print(f"Results in: {OUTPUT_DIR}")
    print(f"Checkpoints in: {CHECKPOINT_DIR}")

In [ ]:
# ============================================================
# INFERENCE: Predict on individual images
# ============================================================

import torch
import cv2
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from src.models.isic_module import ISICLitModule
from src.data.components.transforms import get_val_transforms
from src.ensemble_predict import (
    ensemble_predict, print_results,
    find_best_checkpoint, load_model_from_checkpoint, get_model_img_size,
)

# ---- CONFIGURATION ----
# Option 1: Single model inference
INFERENCE_MODEL = "efficientnet_b0"  # Model to use
INFERENCE_FOLD = 0                    # Fold to use (or set to None for all folds)

# Option 2: Ensemble inference (set to True to use all available models/folds)
USE_ENSEMBLE = False
ENSEMBLE_MODELS = ["efficientnet_b0"]  # Models for ensemble
ENSEMBLE_STRATEGY = "soft"             # "soft" or "hard_weighted"

# Image paths to predict on
IMAGE_PATHS = [
    # Add your image paths here, e.g.:
    # "/path/to/image1.jpg",
    # "/path/to/image2.jpg",
]

# ---- RUN PREDICTIONS ----
if not IMAGE_PATHS:
    print("\u26a0\ufe0f  No image paths specified. Add paths to IMAGE_PATHS list above.")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    if USE_ENSEMBLE:
        # Ensemble prediction across multiple models and folds
        # img_size is auto-detected per model from the TIMM backbone config
        print(f"\nRunning ensemble inference with models: {ENSEMBLE_MODELS}")
        print(f"Strategy: {ENSEMBLE_STRATEGY}")
        results = ensemble_predict(
            models=ENSEMBLE_MODELS,
            checkpoint_dir=CHECKPOINT_DIR,
            image_paths=IMAGE_PATHS,
            img_size=None,  # auto-detect per model
            device=device,
            strategy=ENSEMBLE_STRATEGY,
        )
        print_results(results)
        
    else:
        # Single model prediction
        if INFERENCE_FOLD is not None:
            # Use specific fold
            model_dir = os.path.join(CHECKPOINT_DIR, INFERENCE_MODEL)
            ckpt_path = find_best_checkpoint(model_dir, INFERENCE_FOLD)
        else:
            # Ensemble across all folds of one model
            results = ensemble_predict(
                models=[INFERENCE_MODEL],
                checkpoint_dir=CHECKPOINT_DIR,
                image_paths=IMAGE_PATHS,
                img_size=None,  # auto-detect
                device=device,
            )
            print_results(results)
            ckpt_path = None  # Skip single model display below
        
        if ckpt_path:
            print(f"Loading checkpoint: {ckpt_path}")
            model = load_model_from_checkpoint(ckpt_path, device)
            
            # Auto-detect image size from the model's TIMM backbone
            img_size = get_model_img_size(model)
            threshold = model.best_threshold.item()
            print(f"Auto-detected image size: {img_size}")
            print(f"Optimal threshold: {threshold:.4f}")
            
            transform = get_val_transforms(img_size)
            
            # Predict on each image
            n_images = len(IMAGE_PATHS)
            cols = min(n_images, 4)
            rows = (n_images + cols - 1) // cols
            fig, axes = plt.subplots(rows, cols, figsize=(5 * cols, 5 * rows))
            if n_images == 1:
                axes = [axes]
            else:
                axes = axes.flatten() if hasattr(axes, 'flatten') else [axes]
            
            for idx, img_path in enumerate(IMAGE_PATHS):
                # Load and preprocess
                image = cv2.imread(img_path)
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                transformed = transform(image=image_rgb)["image"].unsqueeze(0).to(device)
                
                # Predict
                with torch.no_grad():
                    logit = model(transformed).squeeze()
                    prob = torch.sigmoid(logit).item()
                    pred = "MALIGNANT" if prob >= threshold else "BENIGN"
                
                # Display
                axes[idx].imshow(image_rgb)
                color = "red" if pred == "MALIGNANT" else "green"
                axes[idx].set_title(
                    f"{pred}\nProb: {prob:.4f} (thr: {threshold:.4f})", 
                    color=color, fontsize=12, fontweight="bold"
                )
                axes[idx].axis("off")
                print(f"  {Path(img_path).name}: {pred} (prob={prob:.4f})")
            
            # Hide unused axes
            for idx in range(n_images, len(axes)):
                axes[idx].set_visible(False)
            
            plt.tight_layout()
            plt.show()
            
            del model
            if torch.cuda.is_available():
                torch.cuda.empty_cache()


In [ ]:
# ============================================================
# TEST INFERENCE: Generate submission from test-image.hdf5
# ============================================================
# This cell loads all test images + tabular features from the
# competition HDF5 file and metadata CSV, runs ensemble inference
# and produces a submission.csv matching the Kaggle format.
# ============================================================

import torch, os, h5py, cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from src.models.isic_module import ISICLitModule
from src.data.components.transforms import get_val_transforms
from src.data.isic_datamodule import TABULAR_NUM_COLS, ISICDataModule
from src.ensemble_predict import (
    find_best_checkpoint, load_model_from_checkpoint,
    get_model_img_size, discover_folds,
)

# ---- CONFIGURATION ----
SUBMISSION_MODELS = ["efficientnet_b0"]   # Models to ensemble
ENSEMBLE_STRATEGY = "soft"                 # "soft" or "hard_weighted"
BATCH_SIZE = 64
OUTPUT_CSV = os.path.join(OUTPUT_DIR, "submission.csv")

TEST_HDF5 = os.path.join(DATA_DIR, "test-image.hdf5")
TEST_META = os.path.join(DATA_DIR, "test-metadata.csv")
SAMPLE_SUB = os.path.join(DATA_DIR, "sample_submission.csv")

# Verify files exist
for path, label in [(TEST_HDF5, "test HDF5"), (TEST_META, "test metadata")]:
    if os.path.exists(path):
        print(f"\u2705 {label}: {path}")
    else:
        print(f"\u26a0\ufe0f  {label} NOT found: {path}")

# Load test IDs and metadata
if os.path.exists(SAMPLE_SUB):
    sample_df = pd.read_csv(SAMPLE_SUB)
    isic_ids = sample_df["isic_id"].tolist()
elif os.path.exists(TEST_META):
    sample_df = pd.read_csv(TEST_META, low_memory=False)
    isic_ids = sample_df["isic_id"].tolist()
else:
    raise FileNotFoundError("Neither sample_submission.csv nor test-metadata.csv found")
print(f"\nTest samples: {len(isic_ids)}")

# Encode tabular features from test metadata
test_meta_df = pd.read_csv(TEST_META, low_memory=False)
test_meta_df = test_meta_df.set_index('isic_id').loc[isic_ids].reset_index()

# Use the same encoding as ISICDataModule
dm_dummy = ISICDataModule.__new__(ISICDataModule)
test_tabular_raw = dm_dummy._encode_tabular(test_meta_df)
# Self-standardize (approximate; ideal: use training stats)
tab_mean = test_tabular_raw.mean(axis=0)
tab_std = test_tabular_raw.std(axis=0)
tab_std[tab_std < 1e-7] = 1.0
test_tabular = (test_tabular_raw - tab_mean) / tab_std
print(f"Tabular features: {test_tabular.shape}")

# ---- LOAD ALL MODELS/FOLDS ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nDevice: {device}")
model_configs = []

for model_name in SUBMISSION_MODELS:
    model_dir = os.path.join(CHECKPOINT_DIR, model_name)
    folds = discover_folds(model_dir)
    if not folds:
        print(f"\u26a0\ufe0f  No folds for {model_name}")
        continue
    print(f"\n{model_name}: {len(folds)} folds")
    for fold in folds:
        ckpt_path = find_best_checkpoint(model_dir, fold)
        if ckpt_path is None:
            continue
        print(f"  Fold {fold}: {os.path.basename(ckpt_path)}")
        model = load_model_from_checkpoint(ckpt_path, device)
        img_size = get_model_img_size(model)
        threshold = model.best_threshold.item()
        auroc = model.best_auroc.item() if hasattr(model, "best_auroc") else 0.5
        uses_tabular = hasattr(model, 'fusion_head') and model.fusion_head is not None
        print(f"    img_size={img_size}, threshold={threshold:.4f}, auroc={auroc:.4f}, tabular={uses_tabular}")
        model_configs.append((model, img_size, threshold, auroc, uses_tabular))

print(f"\nTotal model-folds: {len(model_configs)}")

# ---- RUN INFERENCE ON HDF5 ----
hdf5 = h5py.File(TEST_HDF5, "r")
all_probs, all_thresholds, all_aurocs = [], [], []

for idx, (model, img_size, threshold, auroc, uses_tabular) in enumerate(model_configs):
    print(f"\nInference {idx+1}/{len(model_configs)} (img_size={img_size}, tabular={uses_tabular})...")
    transform = get_val_transforms(img_size)
    probs = []
    for batch_start in tqdm(range(0, len(isic_ids), BATCH_SIZE), desc="Batches"):
        batch_ids = isic_ids[batch_start : batch_start + BATCH_SIZE]
        batch_images = []
        for isic_id in batch_ids:
            img_bytes = hdf5[isic_id][()]
            img = cv2.imdecode(np.frombuffer(img_bytes, np.uint8), cv2.IMREAD_COLOR)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = transform(image=img)["image"]
            batch_images.append(img)
        batch_tensor = torch.stack(batch_images).to(device)
        if uses_tabular:
            batch_tab = torch.tensor(
                test_tabular[batch_start:batch_start+len(batch_ids)], dtype=torch.float32
            ).to(device)
        else:
            batch_tab = None
        with torch.no_grad():
            logits = model(batch_tensor, batch_tab).squeeze(1)
            batch_probs = torch.sigmoid(logits).cpu().numpy()
        probs.extend(batch_probs.tolist())
    all_probs.append(np.array(probs))
    all_thresholds.append(threshold)
    all_aurocs.append(auroc)
    del model
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

hdf5.close()

# ---- ENSEMBLE ----
all_probs = np.array(all_probs)
if ENSEMBLE_STRATEGY == "hard_weighted":
    weights = np.array(all_aurocs)
    weights = weights / weights.sum() if weights.sum() > 0 else np.ones_like(weights) / len(weights)
    hard = np.array([(p >= t).astype(float) for p, t in zip(all_probs, all_thresholds)])
    final_probs = np.average(hard, axis=0, weights=weights)
    final_threshold = 0.5
else:
    final_probs = np.mean(all_probs, axis=0)
    final_threshold = np.mean(all_thresholds)

final_preds = (final_probs >= final_threshold).astype(int)

# ---- GENERATE SUBMISSION ----
submission = pd.DataFrame({"isic_id": isic_ids, "target": final_preds})
submission.to_csv(OUTPUT_CSV, index=False)

print(f"\n{'='*60}")
print(f"SUBMISSION: {OUTPUT_CSV}")
print(f"{'='*60}")
print(f"  Samples: {len(submission)}")
print(f"  Malignant: {final_preds.sum()} ({final_preds.mean()*100:.2f}%)")
print(f"  Benign: {len(final_preds) - final_preds.sum()}")
print(f"  Strategy: {ENSEMBLE_STRATEGY}")
print(f"\nFirst 10 rows:")
print(submission.head(10).to_string(index=False))
